In [1]:
from collections import OrderedDict
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
from urbansim_templates.models import SmallMultinomialLogitStep
import orca
import os; os.chdir('../')
import warnings; warnings.simplefilter('ignore')

import pandas as pd
# import pandana as pdna
import time
import numpy as np
import matplotlib.pyplot as plt
import os
from functools import reduce

# import matplotlib
# matplotlib.style.use('ggplot')

%matplotlib inline

pd.options.display.max_columns = 80

/home/emma/anaconda/envs/ual-model/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
import seaborn as sns
%matplotlib notebook

import glob

from scripts import datasources, models, variables

In [3]:
# import patsy
# from patsy import dmatrix

In [4]:
from choicemodels import MultinomialLogit, MultinomialLogitResults
from choicemodels import mnl
from choicemodels.tools import MergedChoiceTable 
from scripts import simulation
from urbansim.models import util

In [5]:
mm.initialize()

Loading model step 'TOD_choice'
Loading model step 'WLCM'


In [6]:
m = mm.get_step('TOD_choice')

### Format the simulation input data

In [7]:
chooser_filters = ['worker == 1', 'work_at_home == 0']
query = ' and '.join(chooser_filters)

In [8]:
os.listdir('/home/data/fall_2018/')

['jobs_w_occup.csv',
 'households_v2.csv',
 'CHTS_csv_format',
 'urbansim_data_extract.zip',
 'README.txt',
 'bayarea_walk_edges.csv',
 'parcel_attr.csv',
 'bay_area_tertiary_strongly_edges.csv',
 'synthetic_population_data_for_emma.zip',
 'MTC_craigslist_listings_7-10-18.csv',
 'establishments_v2.csv',
 'rentals_with_nodes.csv',
 'jobs_v2.csv',
 'bay_area_tertiary_strongly_nodes.csv',
 '09_01_2015_parcels_v2',
 'persons_v2.csv',
 'persons_w_jobs.csv',
 'bayarea_walk_nodes.csv',
 'area_per_job.csv',
 'building_types.csv',
 'units_v2.csv',
 'persons_v3.csv',
 'physsim-network-nodes.csv',
 'mtc_skims',
 'chts_persons_w_zone_ids.csv',
 'buildings_v2.csv',
 'physsim-network-links.csv',
 'job_sector_ids_v2.csv']

In [9]:
obs = pd.read_csv('/home/data/fall_2018/persons_w_jobs.csv')

In [10]:
jobs = pd.read_csv('/home/data/fall_2018/jobs_v2.csv')

hh = pd.read_csv('/home/data/fall_2018/households_v2.csv')

buildings = pd.read_csv('/home/data/fall_2018/buildings_v2.csv')
parcels = pd.read_csv('/home/data/fall_2018/parcel_attr.csv')

merge = buildings.merge(parcels,how = 'left',left_on='parcel_id', right_on='primary_id')

jobs = jobs.merge(merge,on = 'building_id',how = 'left').rename(columns={'zone_id': 'zone_id_work'})

hh = hh.merge(merge,on = 'building_id',how = 'left').rename(columns={'zone_id': 'zone_id_home'})

In [11]:
obs = obs.merge(jobs,on = 'job_id',how = 'left')

obs = obs.merge(hh,on = 'household_id',how = 'left')

obs.head()

,person_id,member_id,age,primary_commute_mode,relate,edu,sex,hours,hispanic,earning,race_id,student,work_at_home,worker,household_id,node_id_small,node_id_walk,job_id,building_id_x,sector_id,occupation_id,parcel_id_x,development_type_id_x_x,improvement_value_x,residential_units_x,residential_sqft_x,sqft_per_unit_x,non_residential_sqft_x,building_sqft_x,nonres_rent_per_sqft_x,res_price_per_sqft_x,stories_x,year_built_x,redfin_sale_price_x,redfin_sale_year_x,redfin_home_type_x,costar_property_type_x,costar_rent_x,building_type_id_x,primary_id_x,...,recent_mover,block_group_id,single_family,unit_id,building_id_y,parcel_id_y,development_type_id_x_y,improvement_value_y,residential_units_y,residential_sqft_y,sqft_per_unit_y,non_residential_sqft_y,building_sqft_y,nonres_rent_per_sqft_y,res_price_per_sqft_y,stories_y,year_built_y,redfin_sale_price_y,redfin_sale_year_y,redfin_home_type_y,costar_property_type_y,costar_rent_y,building_type_id_y,primary_id_y,development_type_id_y_y,land_value_y,acres_y,county_id_y,zone_id_home,proportion_undevelopable_y,tax_exempt_status_y,apn_y,parcel_id_local_y,geom_id_y,imputation_flag_y,x_y,y_y,shape_area_y,block_id_y,node_id_y
0,0,1,47,1.0,0,22.0,1,40.0,1,85000.0,1,0,0,1,0,6.546892e+07,5.125306e+09,413751,1834898.0,45.0,15.0,1.56916e+06,19.0,7.055244e+07,0.0,0.0,0.0,1212289.0,1212289.0,10.905878,0.0,3.0,1971.0,NaN,NaN,NaN,Retail (Super Regional Mall),Negotiable,0.0,1569163.0,...,0,60855009011,False,1711366,409174.0,1.54672e+06,1.0,284448.8700,6.0,15096.0,2516.000000,0.0,15096.000000,0.0,10.343517,2.0,1965.0,985000.0,2005.0,Single Family Residential,NaN,NaN,1.0,1546718.0,1.0,426851.1300,0.142286,85.0,557.0,0.0,0.0,46724101,,1.530893e+13,"_, du_zonetarget",-121.882277,37.339126,575.809742,6.085501e+13,6.543004e+07
1,1,1,47,1.0,0,22.0,1,40.0,1,85000.0,1,0,0,1,670,2.861307e+08,5.265128e+09,31870,177900.0,61.0,25.0,1.46229e+06,18.0,0.000000e+00,0.0,0.0,0.0,426880.0,426880.0,15.349687,0.0,1.0,1969.0,NaN,NaN,NaN,Sports & Entertainment,-,0.0,1462294.0,...,0,60855003001,False,1595570,1292478.0,1.39246e+06,2.0,103204.5350,5.0,5900.0,1180.000000,2582.0,8482.000000,0.0,193.929673,2.0,1888.0,NaN,NaN,NaN,Office,-,3.0,1392465.0,2.0,516270.4650,0.382178,85.0,539.0,0.0,0.0,25929029,,7.680105e+12,"_, cs_nrsqft, du_zonetarget",-121.901150,37.334387,1546.620624,6.085500e+13,2.861307e+08
2,2,1,47,1.0,0,22.0,1,40.0,1,85000.0,1,0,0,1,740,2.279076e+09,2.860970e+08,1.25835e+06,1297992.0,61.0,25.0,1.31208e+06,18.0,4.264770e+07,0.0,0.0,0.0,147519.0,19200.0,19.564141,0.0,1.0,1931.0,NaN,NaN,NaN,NaN,NaN,0.0,1312083.0,...,0,60855003001,False,1595613,1507170.0,1.39199e+06,1.0,0.0000,1.0,1844.0,1844.367559,0.0,1844.367559,0.0,131.270024,1.0,1918.0,NaN,NaN,NaN,NaN,NaN,1.0,1391986.0,1.0,0.0000,4.038478,85.0,539.0,0.0,0.0,25912121,,2.103531e+12,"_, res_zone_yrblt, cnty_sq_du, stories1",-121.908472,37.343071,16343.153974,6.085500e+13,2.279076e+09
3,3,1,47,1.0,0,22.0,1,40.0,1,85000.0,1,0,0,1,975,3.969451e+09,6.546673e+07,343693,1579647.0,92.0,23.0,1.54665e+06,7.0,0.000000e+00,0.0,0.0,2046.0,9910.0,9910.0,23.095409,0.0,1.0,1930.0,185000.0,1995.0,Multi-Family (5+ Unit),NaN,NaN,10.0,1546649.0,...,0,60855008002,False,1596643,1293789.0,1.39292e+06,1.0,18997.8525,3.0,2985.0,995.000000,0.0,2985.000000,0.0,279.202661,1.0,2007.0,NaN,NaN,NaN,NaN,NaN,1.0,1392924.0,1.0,10207.1475,0.072154,85.0,558.0,0.0,0.0,25947047,,5.030014e+12,"_, du_zonetarget",-121.899219,37.326159,291.998316,6.085501e+13,6.546673e+07
4,4,1,47,1.0,0,22.0,1,40.0,1,85000.0,1,0,0,1,977,6.561551e+07,6.554853e+07,447704,1382392.0,44.0,43.0,1.71842e+06,14.0,7.576144e+05,0.0,0.0,0.0,23400.0,23400.0,9.961782,0.0,1.0,1979.0,NaN,NaN,NaN,Industrial,-,7.0,1718419.0,...,0,60855008002,False,1595984,484870.0,1.39284e+06,2.0,82196.6886,28.0,33572.0,1199.000000,0.0,33572.000000,0.0,257.134271,1.0,2015.0,165000.0,1997.0,Multi-Family (2-4 Unit),NaN,NaN,3.0,1392836.0,2.0,126477.3114,0.094123,85.0,558.0,0.0,0.0,25945049,,1.158767e+13,"_, du_zonetarget",-121.898518,37.329283,380.903761,6.085501e+13,

In [12]:
obs.index.name = 'obs_id'
obs = obs.query(query)
obs = obs[[
    'zone_id_home', 'zone_id_work',
    'age', 'edu', 'sex','hours','hispanic','race_id',
    'income','persons',
    'sector_id','occupation_id']]

In [13]:
obs.isnull().sum(axis = 0)

zone_id_home     234345
zone_id_work     234345
age                   0
edu                   0
sex                   0
hours                 0
hispanic              0
race_id               0
income                0
persons               0
sector_id        234345
occupation_id    234345
dtype: int64

In [14]:
obs.dropna(inplace = True)

In [15]:
skims = pd.read_csv('/home/emma/ual_model_workspace/fall-2018-models/skims_110118.csv')

obs = pd.merge(obs, skims, how = 'left', left_on=['zone_id_home','zone_id_work'], right_on=['orig','dest'])

obs = pd.merge(obs, skims, how = 'left', left_on=['zone_id_work','zone_id_home'], right_on=['orig','dest'], suffixes=('_HW', '_WH'))

In [16]:
TOD_list = ['EA','AM','MD','PM','EV']

for tod1 in TOD_list:
    for tod2 in TOD_list:
        col_name = f'da_Time_{tod1}_{tod2}'
        obs[col_name] = obs[f'da_Time_{tod1}_HW'] + obs[f'da_Time_{tod2}_WH']

In [17]:
obs = obs[[
    'zone_id_home', 'zone_id_work',
    'age', 'edu', 'sex','hours','hispanic','race_id',
    'income','persons',
    'sector_id','occupation_id',    
    'da_Time_AM_MD',
    'da_Time_AM_PM',
    'da_Time_AM_EV',
    'da_Time_MD_MD',
    'da_Time_MD_PM',
    'da_Time_MD_EV']]

In [18]:
obs['150kplus'] = ((obs['income'] > 150000) | (obs['income'] == 150000)).astype(int)

# obs['no_higher_ed'] = (obs['edu'] < 21).astype(int)

obs['lessGED'] = (obs['edu'] < 16).astype(int)
obs['GED'] = np.where(obs['edu'].isin([16,17]),1,0)
obs['someBach'] = np.where(obs['edu'].isin([18,19]),1,0)
obs['Assoc'] = np.where(obs['edu'].isin([20]),1,0)
obs['Bach'] = np.where(obs['edu'].isin([21]),1,0)

obs['female'] = obs['sex'] - 1

obs['white'] = np.where(obs['race_id'].isin([1.0]),1,0)
obs['minority'] = np.where(obs['white'] == 1, 0, 1)
obs['16less25'] = np.where(((obs.age.between(16,25,inclusive = False)) | (obs.age==16)),1,0)
obs['1per'] = np.where(obs.persons.isin([1.0]),1,0)

In [19]:
# industry of alternatives
obs['sector_retail'] = obs['sector_id'].isin([44, 45]).astype(int)
obs['health'] = obs['sector_id'].isin([62]).astype(int)
obs['info'] = obs['sector_id'].isin([51]).astype(int)
obs['scitech'] = obs['sector_id'].isin([54]).astype(int)
# alts['sector_food_and_hosp'] = alts['sector_id'].isin([72]).astype(int)
obs['manufacturing'] = obs['sector_id'].isin([31, 32, 33]).astype(int)
obs['educ'] = obs['sector_id'].isin([61]).astype(int)
obs['services'] = obs['sector_id'].isin([81]).astype(int)
obs['construction'] = obs['sector_id'].isin([23]).astype(int)
obs['gov'] = obs['sector_id'].isin([92]).astype(int)
obs['finance'] = obs['sector_id'].isin([52]).astype(int)
# obs['sector_whlsale'] = obs['sector_id'].isin([42]).astype(int)
# alts['sector_admin'] = alts['sector_id'].isin([56]).astype(int)
# alts['sector_transport'] = alts['sector_id'].isin([48]).astype(int)
# alts['sector_arts'] = alts['sector_id'].isin([71]).astype(int)
# alts['sector_util'] = alts['sector_id'].isin([22]).astype(int)

# # occupation of alternatives
# alts['occup_mgmt'] = alts['occupation_id'].isin([11]).astype(int)
# alts['occup_sales'] = alts['occupation_id'].isin([41]).astype(int)
# alts['occup_biz'] = alts['occupation_id'].isin([13]).astype(int)
# alts['occup_admin'] = alts['occupation_id'].isin([43]).astype(int)
# alts['occup_edu'] = alts['occupation_id'].isin([25]).astype(int)
# alts['occup_food'] = alts['occupation_id'].isin([35]).astype(int)
# alts['occup_health'] = alts['occupation_id'].isin([29, 31]).astype(int)
# alts['occup_tech'] = alts['occupation_id'].isin([15]).astype(int)
# alts['occup_eng'] = alts['occupation_id'].isin([17]).astype(int)
# alts['occup_transp'] = alts['occupation_id'].isin([53]).astype(int)
# alts['occup_constr'] = alts['occupation_id'].isin([47]).astype(int)

In [20]:
# trips['manager'] = np.where(trips['OCCUP'].isin([11]),1,0)
# trips['business_finance'] = np.where(trips['OCCUP'].isin([13]),1,0)
# trips['computer_math'] = np.where(trips['OCCUP'].isin([15]),1,0)
# trips['arch_eng'] = np.where(trips['OCCUP'].isin([17]),1,0)
# trips['scientist'] = np.where(trips['OCCUP'].isin([19]),1,0)
# trips['socialservice'] = np.where(trips['OCCUP'].isin([21]),1,0)
# trips['legal'] = np.where(trips['OCCUP'].isin([23]),1,0)
# trips['teacher'] = np.where(trips['OCCUP'].isin([25]),1,0)
# trips['arts_entertain'] = np.where(trips['OCCUP'].isin([27]),1,0)
# trips['healthcare_practitioner'] = np.where(trips['OCCUP'].isin([29]),1,0)
# trips['healthcare_support'] = np.where(trips['OCCUP'].isin([31]),1,0)
# trips['protectiveservice'] = np.where(trips['OCCUP'].isin([33]),1,0)
# trips['foodprep'] = np.where(trips['OCCUP'].isin([35]),1,0)
# trips['cleaning_maintenance'] = np.where(trips['OCCUP'].isin([37]),1,0)
# trips['personalcare'] = np.where(trips['OCCUP'].isin([39]),1,0)
# trips['sales'] = np.where(trips['OCCUP'].isin([41]),1,0)
# trips['office_admin'] = np.where(trips['OCCUP'].isin([43]),1,0)
# trips['farmer'] = np.where(trips['OCCUP'].isin([45]),1,0)
# trips['construction_extraction'] = np.where(trips['OCCUP'].isin([47]),1,0)
# trips['installation_mechanic'] = np.where(trips['OCCUP'].isin([49]),1,0)
# trips['production'] = np.where(trips['OCCUP'].isin([51]),1,0)
# trips['transporter'] = np.where(trips['OCCUP'].isin([53]),1,0)
# trips['military'] = np.where(trips['OCCUP'].isin([55]),1,0)
# trips['other'] = np.where(trips['OCCUP'].isin([97]),1,0)

# trips['agri'] = np.where(trips['INDUS'].isin([11]),1,0)
# trips['mining'] = np.where(trips['INDUS'].isin([21]),1,0)
# trips['utility'] = np.where(trips['INDUS'].isin([22]),1,0)
# trips['construction'] = np.where(trips['INDUS'].isin([23]),1,0)
# trips['manufacturing'] = np.where(trips['INDUS'].isin([31]),1,0)
# trips['trade'] = np.where(trips['INDUS'].isin([42,44]),1,0)
# trips['transport'] = np.where(trips['INDUS'].isin([48]),1,0)
# trips['info'] = np.where(trips['INDUS'].isin([51]),1,0)
# trips['finance'] = np.where(trips['INDUS'].isin([52]),1,0)
# trips['realestate'] = np.where(trips['INDUS'].isin([53]),1,0)
# trips['scitech'] = np.where(trips['INDUS'].isin([54]),1,0)
# trips['manage'] = np.where(trips['INDUS'].isin([55]),1,0)
# trips['admin'] = np.where(trips['INDUS'].isin([56]),1,0)
# trips['educ'] = np.where(trips['INDUS'].isin([61]),1,0)
# trips['health'] = np.where(trips['INDUS'].isin([62]),1,0)
# trips['artsrec'] = np.where(trips['INDUS'].isin([71]),1,0)
# trips['hospitality'] = np.where(trips['INDUS'].isin([72]),1,0)
# trips['services'] = np.where(trips['INDUS'].isin([81]),1,0)
# trips['gov'] = np.where(trips['INDUS'].isin([92]),1,0)
# trips['other'] = np.where(trips['INDUS'].isin([97]),1,0)

In [21]:
len(obs)

3060996

In [22]:
import random

In [51]:
# k = int(round(len(obs) * (2346/4681)))

# data= np.full((k, 1), 13)

# #Creating pandas dataframe from numpy array
# b = pd.DataFrame({'TOD':data[:,0]})

b = pd.DataFrame({'TOD':(np.full((int(round(len(obs) * (2346/4681))), 1), 13)[:,0])})

c = pd.DataFrame({'TOD':(np.full((int(round(len(obs) * (573/4681))), 1), 12)[:,0])})

d = pd.DataFrame({'TOD':(np.full((int(round(len(obs) * (420/4681))), 1), 23)[:,0])})

e = pd.DataFrame({'TOD':(np.full((int(round(len(obs) * (404/4681))), 1), 14)[:,0])})

f = pd.DataFrame({'TOD':(np.full((int(round(len(obs) * (377/4681))), 1), 24)[:,0])})

g = pd.DataFrame({'TOD':(np.full((int(round(len(obs) * (201/4681))), 1), 2)[:,0])})

h = pd.DataFrame({'TOD':(np.full((int(round(len(obs) * (196/4681))), 1), 3)[:,0])})

j = pd.DataFrame({'TOD':(np.full((int(round(len(obs) * (164/4681))), 1), 22)[:,0])})

In [52]:
frames = [b,c,d,e,f,g,h,j]

alts = pd.concat(frames)

len(alts)

3060997

In [ ]:
# 13.0    2346
# 12.0     573
# 23.0     420
# 14.0     404
# 24.0     377
# 2.0      201
# 3.0      196
# 22.0     164

In [53]:
alts['EA_MD'] = alts['TOD'].isin([2]).astype(int)
alts['EA_PM'] = alts['TOD'].isin([3]).astype(int)
alts['AM_MD'] = alts['TOD'].isin([12]).astype(int)
alts['AM_PM'] = alts['TOD'].isin([13]).astype(int)
alts['AM_EV'] = alts['TOD'].isin([14]).astype(int)
alts['MD_MD'] = alts['TOD'].isin([22]).astype(int)
alts['MD_PM'] = alts['TOD'].isin([23]).astype(int)
alts['MD_EV'] = alts['TOD'].isin([24]).astype(int)

## Simulation with constrained choices

In [55]:
model = MultinomialLogitResults(model_expression = m.model_expression, 
                    fitted_parameters = m.fitted_parameters)

AttributeError: 'SmallMultinomialLogitStep' object has no attribute 'fitted_parameters'

In [56]:
def mct_callable(obs, alts):
    return MergedChoiceTable(obs, alts, sample_size=m.alt_sample_size 
#                              ,interaction_terms=[interaction_terms_tt, interaction_terms_dist, interaction_terms_cost]
                            )
        
def probs_callable(mct):
    return model.probabilities(mct)

### Iterative Lottery Choices with Sampling

In [57]:
choices = simulation.iterative_lottery_choices(
    obs, alts, mct_callable=mct_callable, probs_callable=probs_callable, alt_capacity=m.alt_capacity,
    chooser_size=m.chooser_size, max_iter=m.max_iter, chooser_batch_size=200000)

AttributeError: 'SmallMultinomialLogitStep' object has no attribute 'alt_capacity'

In [ ]:
print("TODM placed {0} of {1} workers {2}% during simulation.".format(
    len(choices), len(obs), np.round(len(choices)/len(obs) * 100, 1)))

## Save Results

In [ ]:
# persons = orca.get_table('persons')

In [ ]:
# persons.update_col('TOD', pd.Series())

In [ ]:
# persons.update_col_from_series('TOD', choices, cast=True)

In [ ]:
# persons.to_frame().to_csv('./data/persons_w_TOD_2018_12_06.csv')